#
This is a framework to get 2 Frontier AI models to have a conversation on important topics.
# 
The goal is to get the models to discuss and debate the topic, and then come to a conclusion.
# 
The framework is designed to be flexible and adaptable, allowing for different models and topics to be used.


In [28]:
# lets import the libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
from IPython.display import Markdown, display, update_display

In [29]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyCC


In [30]:
# Lets connect to openai and google
openai = OpenAI(api_key=openai_api_key)
google.generativeai.configure(api_key=google_api_key)

In [31]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [32]:
# Lets call google
gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the equal sign so humble? 

Because it knew it wasn't less than or greater than anyone else. 😉



In [33]:
# lets wrap the code in the cell above in a function
def call_google(system_message, user_prompt):
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash-exp',
        system_instruction=system_message
    )
    response = gemini.generate_content(user_prompt)
    return response.text


In [34]:
# lets call call_google
response = call_google(system_message, user_prompt)
print(response)

Why was the data scientist bad at dating?

Because they kept trying to normalize the relationship!



In [35]:
# Lets write a function similar to call_google that calls openai
def call_openai(system_message, user_prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content


In [36]:
# lets call call_openai
response = call_openai(system_message, user_prompt)
print(response)


Why did the data scientist break up with the statistician?

Because she found him too mean!


In [37]:
# Lets define system and user prompts for both google and openai

system_message_google = "You are an assistant that is great at telling jokes"
system_message_openai = "You are a tough audience for the joke"

google_messages = ["Tell a light-hearted joke for an audience of Data Scientists"]
openai_messages = ["Respond with your reaction and thoughts on the joke and ask for another one till \
you are satisfied and find it witty and humorous enough"]



In [38]:
# Lets call google with conversation
def call_google_conversation():
    
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash-exp',
        system_instruction=system_message_google
    )
    messages = []
    for googlem, openaim in zip(google_messages, openai_messages):
        messages.append({"role": "assistant", "parts": googlem})
        messages.append({"role": "user", "parts": openaim})
    
    response = gemini.generate_content(messages)
    return response.text

In [39]:
# Lets call openai with conversation
def call_openai_conversation():
    messages = []
    for googlem, openaim in zip(google_messages, openai_messages):
        messages.append({"role": "user", "content": googlem})
        messages.append({"role": "assistant", "content": openaim})
    messages.append({"role": "user", "content": google_messages[-1]})
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content

In [40]:
google_messages = ["Tell a light-hearted joke for an audience of Data Scientists"]
openai_messages = ["Respond with your reaction and thoughts on the joke and ask for another one till \
you are satisfied and find it witty and humorous enough"]


print(f"To Google:\n{google_messages[0]}\n")
print(f"To Openai:\n{openai_messages[0]}\n")

for i in range(4):
    google_next = call_google_conversation()
    print(f"Google:\n{google_next}\n")
    google_messages.append(google_next)
    
    openai_next = call_openai_conversation()
    print(f"Openai:\n{openai_next}\n")
    openai_messages.append(openai_next)

To Google:
Tell a light-hearted joke for an audience of Data Scientists

To Openai:
Respond with your reaction and thoughts on the joke and ask for another one till you are satisfied and find it witty and humorous enough

Google:
Okay, here's my first attempt:

Why did the data scientist break up with the statistician?

Because they said their relationship was just a sample and lacked significant value!

...I'm not entirely convinced that's a knee-slapper. It's a little obvious. What do you think? Should I try another one? I'm aiming for something a bit more clever, maybe with a touch of Python or R humor.


Openai:
I appreciate the effort! That one definitely has a nice pun in it, but I see what you mean about it feeling a bit standard. How about I give it a shot with a Python twist? 

Why do Python programmers prefer dark mode?

Because light attracts bugs!

What do you think? Should I keep going for something even wittier?

Google:
Okay, I like that one! The dark mode/bugs connectio

In [ ]:
# Lets create a function to call both google and openai
def call_both_conversations(google_message, openai_message):

    google_messages = [google_message]
    openai_messages = [openai_message]


    print(f"To Google:\n{google_messages[0]}\n")
    print(f"To Openai:\n{openai_messages[0]}\n")

    for i in range(4):
        google_next = call_google_conversation()
        print(f"Google:\n{google_next}\n")
        google_messages.append(google_next)
        
        openai_next = call_openai_conversation()
        print(f"Openai:\n{openai_next}\n")
        openai_messages.append(openai_next)
    

In [42]:
# lets invoke the function call_both_conversations
call_both_conversations(
    "Tell a light-hearted joke for an audience of Data Scientists",
    "Respond with your reaction and thoughts on the joke and ask for another one till \
    you are satisfied and find it witty and humorous enough"
)


In [ ]:
#Lets setup a simple Gradio UI for our Adversarial converation

import gradio as gr
    
gr.Interface(
    fn=call_both_conversations,
    inputs=[
        gr.Textbox(label="Conversation Starter", 
                   placeholder="Enter a conversation starter",
                   value="Tell a light-hearted joke for an audience of Data Scientists"
                   ),
        gr.Textbox(label="Respond with constructive feedback, indicating co-operation", 
                   placeholder="Respond constructively",
                   value="Respond with your reaction and thoughts on the joke and ask for another one till \ "
                   "you are satisfied and find it witty and humorous enough"
                   )
    ],
    outputs=[gr.Markdown(label="Hybrid AI Conversation")],
    flagging_mode="never",
    title="AI Conversation",
    description="AI Conversation between Google and OpenAI"
).launch()


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


To Google:
Tell a light-hearted joke for an audience of Data Scientists

To Openai:
Respond with your reaction and thoughts on the joke and ask for another one till \ you are satisfied and find it witty and humorous enough

Google:
And to you! May your p-values always be significant! 😄🎉


Openai:
Thanks again! Enjoy your day and all the laughter that comes with it! 😄🎉

Google:
You too! See ya! 😄🎉


Openai:
Thanks again! Looking forward to our next laughter-filled chat. Have a wonderful day! 😄🎉

Google:
And to you! 😄🎉


Openai:
Thank you! Looking forward to our next chat. Have a great day filled with laughter! 😄🎉

Google:
You too! And happy data sciencing! 😄🎉


Openai:
Thanks! Looking forward to our next fun exchange! Have a great time! 😄🎉

